In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree
from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [27]:
# getting data from full dataset
dat = pd.read_excel("../raw_data/airport_choice_survey_EN_ver2.0_Capstone.xlsx",sheet_name='Data', header =0)
income = dat.loc[:,["ID","Income"]]
income

,ID,Income
0,1,5.0
1,2,3.0
2,3,NaN
3,4,7.0
4,5,3.0
5,6,3.0
6,7,3.0
7,8,3.0
8,9,NaN
9,10,NaN


In [59]:
# getting data from full dataset
cldata = pd.read_csv("../data/Cleaned_Data.csv")
cldata
#drop unused columns
cldata = cldata.drop(columns =["FrequentDestination1",
                             "FrequentDestination2",
                             "FrequentDestination3",
                             "FrequentDestination4",
                             "FrequentDestination5",
                             "FrequentDestination6",
                             "FrequentDestination7",
                        "TotalDepartureHr"], axis =1)

In [29]:
dat.loc[:,"Income"].unique()

array([ 5.,  3., nan,  7.,  2.,  4.,  1.,  6.])

In [30]:
merdf = pd.merge(cldata, income, how='inner', on = 'ID')
merdf.head(5)

,ID,Airport,Airline,Age,Gender,Nationality,TripPurpose,TripDuration,FlyingCompanion,ProvinceResidence,...,AccessCost,AccessTime,Occupation,MileageAirline,MileageAirline1,MileageAirline2,MileageAirline3,MileageAirline4,MileageAirline5,Income
0,1,1,1,49,1,1,2,7,0,3,...,8000.0,40,1,1,1,0,0,0,0,5.0
1,2,1,1,49,2,1,1,4,4,3,...,8000.0,50,9,5,0,0,0,0,1,3.0
2,3,1,1,25,1,1,1,10,2,3,...,1000.0,20,12,5,0,0,0,0,1,NaN
3,4,1,1,29,1,1,2,7,2,3,...,12642.0,76,8,2,0,1,0,0,0,7.0
4,5,1,1,34,2,1,2,4,0,3,...,8000.0,50,1,5,0,0,0,0,1,3.0


In [47]:
treedf = merdf.drop(columns =["ID","Airport","Airline"], axis =1)

In [48]:
df = treedf[treedf.loc[:,'Income'].notna()]
validata= merdf[merdf.loc[:,'Income'].isna()]

In [33]:
X=  df.loc[:, ~df.columns.isin(['Income','ID'])]
y= df['Income'].astype('category').cat.codes
y.name='Income'
# train airport model with default parameter
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=6)

In [34]:
y


0      4
1      2
3      6
4      2
5      2
6      2
7      2
11     1
12     1
13     3
14     2
18     2
20     0
21     1
22     3
23     2
24     2
29     0
30     3
31     4
35     1
38     3
39     3
41     4
44     1
46     1
47     2
48     2
49     3
52     1
      ..
430    0
431    0
432    0
433    2
434    1
435    1
436    2
438    6
439    2
441    0
442    3
443    4
445    1
446    2
447    3
448    1
449    1
450    3
451    1
452    2
453    3
454    3
455    2
456    2
457    4
458    0
459    4
460    6
461    6
462    6
Name: Income, Length: 337, dtype: int8

In [35]:
# decision tree function, return trained model
def decision_tree(X_train, y_train, maxdepth = 10, max_feature= None, maxleaf =None, minsamleaf =1, minsamsplit = 2 ):
    clf =tree.DecisionTreeClassifier(class_weight=None, 
                                     criterion ='gini', 
                                     max_depth =maxdepth, 
                                     max_features = max_feature, 
                                     max_leaf_nodes= maxleaf,
                                    min_samples_leaf = minsamleaf, min_samples_split =minsamsplit,
                                    min_weight_fraction_leaf =0.0, presort =False, random_state =100, splitter = 'best')
    clf = clf.fit(X_train, y_train)
    tree.export_graphviz(clf,out_file=str(y_train.name)+".dot",feature_names=X_train.columns)
    return(clf)

In [36]:
def test_decisiont_tree(X_test, y_test, model):
    y_pred =model.predict(X_test)  
    print(metrics.confusion_matrix(y_test,y_pred))
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [37]:
income_default = decision_tree(X_train, y_train,maxdepth =6, max_feature= None, maxleaf =None, minsamleaf =2, minsamsplit = 6)
test_decisiont_tree(X_test,y_test,income_default)

[[ 4  5  3  1  0  0  0]
 [ 6 11  2  6  0  0  0]
 [ 0 11  9  3  1  0  2]
 [ 0  8  6  2  0  1  1]
 [ 0  5  3  1  1  0  0]
 [ 0  5  1  0  1  0  0]
 [ 0  2  0  1  0  0  0]]
Accuracy: 0.2647058823529412
             precision    recall  f1-score   support

          0       0.40      0.31      0.35        13
          1       0.23      0.44      0.31        25
          2       0.38      0.35      0.36        26
          3       0.14      0.11      0.12        18
          4       0.33      0.10      0.15        10
          5       0.00      0.00      0.00         7
          6       0.00      0.00      0.00         3

avg / total       0.26      0.26      0.25       102



In [49]:
x_validata= validata.loc[:, ~validata.columns.isin(['Income',"ID","Airport","Airline"])]
income_pre =income_default.predict(x_validata)

In [50]:
validata['income_pred']=income_pre

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
validata.describe()

,ID,Airport,Airline,Age,Gender,Nationality,TripPurpose,TripDuration,FlyingCompanion,ProvinceResidence,...,AccessTime,Occupation,MileageAirline,MileageAirline1,MileageAirline2,MileageAirline3,MileageAirline4,MileageAirline5,Income,income_pred
count,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.00000,126.000000,126.000000,126.000000,...,126.000000,126.000000,126.000000,126.000000,126.0,126.000000,126.000000,126.000000,0.0,126.000000
mean,152.547619,1.166667,2.388889,37.928571,1.515873,1.452381,1.47619,25.579365,2.825397,3.539683,...,64.246032,10.317460,4.785714,0.031746,0.0,0.015873,0.055556,0.896825,NaN,0.777778
std,104.396828,0.374166,1.193128,12.897242,0.501743,1.047719,0.84583,56.585419,4.108682,2.368631,...,69.513934,3.383846,0.765320,0.176023,0.0,0.125483,0.229976,0.305401,NaN,1.225652
min,3.000000,1.000000,1.000000,17.000000,1.000000,1.000000,1.00000,0.000000,0.000000,1.000000,...,5.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,NaN,0.000000
25%,77.250000,1.000000,1.000000,28.000000,1.000000,1.000000,1.00000,5.000000,1.000000,1.000000,...,25.000000,12.000000,5.000000,0.000000,0.0,0.000000,0.000000,1.000000,NaN,0.000000
50%,140.500000,1.000000,2.000000,36.500000,2.000000,1.000000,1.00000,6.000000,2.000000,3.000000,...,50.000000,12.000000,5.000000,0.000000,0.0,0.000000,0.000000,1.000000,NaN,0.000000
75%,202.750000,1.000000,3.750000,45.000000,2.000000,1.000000,2.00000,10.000000,3.000000,5.000000,...,60.000000,12.000000,5.000000,0.000000,0.0,0.000000,0.000000,1.000000,NaN,1.000000
max,468.000000,2.000000,4.000000,80.000000,2.000000,5.000000,4.00000,365.000000,26.000000,8.000000,...,390.000000,12.000000,5.000000,1.000000,0.0,1.000000,1.000000,1.000000,NaN,6.000000


In [52]:
print(validata['income_pred'].value_counts()
     )
print('Income class 0%: ', round(validata['income_pred'].value_counts()[0]/validata.shape[0]*100,3))
print('Income class 1%: ', round(validata['income_pred'].value_counts()[1]/validata.shape[0]*100,3))
print('Income class 2%: ', round(validata['income_pred'].value_counts()[2]/validata.shape[0]*100,3))
print('Income class 3%: ', round(validata['income_pred'].value_counts()[3]/validata.shape[0]*100,3))
print('Income class 4%: ', round(validata['income_pred'].value_counts()[4]/validata.shape[0]*100,3))
print('Income class 6%: ', round(validata['income_pred'].value_counts()[6]/validata.shape[0]*100,3))

0    72
1    30
2    14
3     6
6     3
4     1
Name: income_pred, dtype: int64
Income class 0%:  57.143
Income class 1%:  23.81
Income class 2%:  11.111
Income class 3%:  4.762
Income class 4%:  0.794
Income class 6%:  2.381


In [80]:
output = validata.loc[:,["ID","income_pred"]]
output.to_csv('output.csv',header=True,index=False)
validate = cldata[cldata.ID.isin(output.loc[:,"ID"])]

In [85]:
outp = pd.merge(validate, output, how='inner', on = 'ID')
#validate.to_csv('output.csv',header=True,index=False)
#validate["compare"] = 
outp[outp.Income != outp.income_pred]

,ID,Airport,Airline,Age,Gender,Nationality,TripPurpose,TripDuration,FlyingCompanion,ProvinceResidence,...,AccessTime,Occupation,Income,MileageAirline,MileageAirline1,MileageAirline2,MileageAirline3,MileageAirline4,MileageAirline5,income_pred
0,3,1,1,25,1,1,1,10,2,3,...,20,12,3,5,0,0,0,0,1,0
1,9,1,3,24,2,1,1,4,1,3,...,5,12,3,5,0,0,0,0,1,0
2,10,1,1,52,2,1,1,5,1,5,...,60,12,2,5,0,0,0,0,1,0
3,11,1,1,29,2,1,1,5,2,3,...,10,12,3,5,0,0,0,0,1,0
4,16,1,3,31,2,1,1,5,2,3,...,5,12,2,5,0,0,0,0,1,0
5,17,1,3,28,2,1,1,4,1,3,...,10,12,3,5,0,0,0,0,1,0
6,18,1,3,29,2,1,1,6,5,3,...,20,12,3,5,0,0,0,0,1,0
7,20,1,3,29,1,1,1,6,5,3,...,76,2,3,5,0,0,0,0,1,1
8,26,1,4,22,1,4,4,30,0,3,...,15,12,3,5,0,0,0,0,1,0
9,27,1,1,55,1,1,1,5,3,3,...,10,12,2,5,0,0,0,0,1,0
